In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [5]:
!head data/RealEstate.csv

In [11]:
schema = 'MLS string, Location string, Price double, Bedrooms int, Bathrooms int, Size double, PriceSqFt double, Status string'
df = spark.read.csv('data/RealEstate.csv', sep=',', header=True, schema=schema)

In [57]:
# Average price using groupBy + avg
df.groupBy('Bedrooms') \
  .avg('Price') \
  .rdd \
  .map(lambda r: (r['Bedrooms'], r['avg(Price)'])) \
  .collect()

[(1, 169981.81818181818),
 (6, 603225.0),
 (3, 359062.20649651974),
 (5, 657858.0645161291),
 (4, 483475.6497175141),
 (7, 325000.0),
 (10, 699000.0),
 (2, 266356.3739837398),
 (0, 293450.0)]

In [55]:
# Average price using reduceByKey + mapValues
df.rdd \
  .map(lambda h: (h.Bedrooms, (h.Price, 1))) \
  .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \
  .mapValues(lambda v: v[0]/v[1]) \
  .collect()

[(0, 293450.0),
 (1, 169981.81818181818),
 (2, 266356.3739837398),
 (3, 359062.20649651974),
 (4, 483475.6497175141),
 (5, 657858.0645161291),
 (6, 603225.0),
 (7, 325000.0),
 (10, 699000.0)]